In [ ]:
from gensim.models import Word2Vec
import pandas as pd
from sklearn.manifold import TSNE
import os
import numpy as np
import sys

# Installation:
# https://pypi.org/project/fitsne/0.1.10/
# I *also* downloaded fftw and installed using
# http://fftw.org/install/mac.html with commands below
# $./configure --enable-threads && make
# $ sudo make install
# I'm unsure if that's needed, or if the pip commands listed on the pypi link 
# are enough to get it working. Things seem to execute on multiple cores


# You NEED to clone https://github.com/KlugerLab/FIt-SNE
# Then you NEED to execute
# g++ -std=c++11 -O3  src/sptree.cpp src/tsne.cpp src/nbodyfft.cpp  -o bin/fast_tsne -pthread -lfftw3 -lm
# Then you NEED to add the repository path (which will have your compiled code in ./bin) to sys
# As shown on the next line
sys.path.append('/Users/danielklass/Dropbox/GaTech/cse6242_project/FIt-SNE')

from fast_tsne import fast_tsne
RAND = 42

In [ ]:
workers = os.cpu_count() - 2

In [ ]:
MOVIE_ID = 'movieId'
TITLE = 'title'
VECTOR = 'vector'

In [ ]:
model = Word2Vec.load(os.path.join(
    'gensim_models2/w2v_vs_16_sg_1_hs_1_mc_1_it_1_wn_32_ng_2.gensim'))

In [ ]:
df_movies = pd.read_csv('ml-20m/movies.csv', index_col=MOVIE_ID)

In [ ]:
def get_movie_vector(i):
    try:
        return model.wv.get_vector(str(i))
    except KeyError:
        return np.nan

In [ ]:
df_movies[VECTOR] = df_movies.index.get_level_values(MOVIE_ID).map(get_movie_vector)

In [ ]:
df_movies = df_movies[pd.notnull(df_movies[VECTOR])]

In [ ]:
vectors = df_movies[VECTOR].to_numpy()

In [ ]:
vectors = np.vstack(vectors)

In [ ]:
X_tsne = fast_tsne(vectors, seed=RAND, nthreads=workers)

In [ ]:
# X_tSNE = TSNE(n_components=2, random_state=RAND).fit_transform(vectors[0:200])